In [6]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# Create Orders table
n_orders = 10000
orders = {
    'OrderID': range(1, n_orders + 1),
    'OrderDate': [datetime(2024, 1, 1) + timedelta(days=np.random.randint(0, 365)) for _ in range(n_orders)]
}
df_orders = pd.DataFrame(orders)

# Create Order_Items table
products = [
    (1, 'Milk'), (2, 'Bread'), (3, 'Eggs'), (4, 'Butter'), (5, 'Cheese'),
    (6, 'Cereal'), (7, 'Coffee'), (8, 'Tea'), (9, 'Sugar'), (10, 'Jam')
]
n_items = 30000  # Total items across orders
order_items = {
    'OrderID': np.random.choice(df_orders['OrderID'], n_items),
    'ProductID': np.random.choice([p[0] for p in products], n_items),
}
df_items = pd.DataFrame(order_items)
# Add ProductName
df_items = df_items.merge(pd.DataFrame(products, columns=['ProductID', 'ProductName']), on='ProductID')

# Create SQLite database
conn = sqlite3.connect('retail.db')
df_orders.to_sql('Orders', conn, if_exists='replace', index=False)
df_items.to_sql('Order_Items', conn, if_exists='replace', index=False)

# Verify tables
print("Orders table:")
print(pd.read_sql("SELECT * FROM Orders LIMIT 5", conn))
print("\nOrder_Items table:")
print(pd.read_sql("SELECT * FROM Order_Items LIMIT 5", conn))

conn.close()
print("Database 'retail.db' created with Orders and Order_Items tables.")

Orders table:
   OrderID            OrderDate
0        1  2024-04-12 00:00:00
1        2  2024-12-14 00:00:00
2        3  2024-09-27 00:00:00
3        4  2024-04-16 00:00:00
4        5  2024-03-12 00:00:00

Order_Items table:
   OrderID  ProductID ProductName
0     8594          3        Eggs
1     2654          5      Cheese
2     5612          8         Tea
3     6128          1        Milk
4     3924          2       Bread
Database 'retail.db' created with Orders and Order_Items tables.


In [10]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('retail.db')

# Execute the SQL query and fetch the result
query = "SELECT COUNT(DISTINCT OrderID) AS TotalTransactions FROM Orders"
result = conn.execute(query).fetchone()

# Print the result
print("Total Transactions:", result[0])

# Close the connection
conn.close()

Total Transactions: 10000


In [11]:
import sqlite3
conn = sqlite3.connect('retail.db')
# Query 1: Total transactions
total_tx = pd.read_sql("SELECT COUNT(DISTINCT OrderID) AS TotalTransactions FROM Orders", conn)
print("Total Transactions:", total_tx)
# Query 2: Frequent pairs
pairs = pd.read_sql("""
    WITH ProductPairs AS (
        SELECT 
            a.ProductName AS ProductA,
            b.ProductName AS ProductB,
            COUNT(DISTINCT a.OrderID) AS PairCount
        FROM Order_Items a
        JOIN Order_Items b ON a.OrderID = b.OrderID AND a.ProductID < b.ProductID
        GROUP BY a.ProductName, b.ProductName
        HAVING PairCount >= 100
    )
    SELECT ProductA, ProductB, PairCount
    FROM ProductPairs
    ORDER BY PairCount DESC
    LIMIT 5
""", conn)
print("\nTop 5 Frequent Pairs:")
print(pairs)
# Query 3: Support, confidence, lift
metrics = pd.read_sql("""
    WITH ProductPairs AS (
        SELECT 
            a.ProductName AS ProductA,
            b.ProductName AS ProductB,
            COUNT(DISTINCT a.OrderID) AS PairCount
        FROM Order_Items a
        JOIN Order_Items b ON a.OrderID = b.OrderID AND a.ProductID < b.ProductID
        GROUP BY a.ProductName, b.ProductName
    ),
    ProductCounts AS (
        SELECT ProductName, COUNT(DISTINCT OrderID) AS ProductCount
        FROM Order_Items
        GROUP BY ProductName
    ),
    Total AS (
        SELECT COUNT(DISTINCT OrderID) AS TotalTransactions
        FROM Orders
    )
    SELECT 
        pp.ProductA,
        pp.ProductB,
        pp.PairCount,
        (pp.PairCount * 1.0 / TotalTransactions) AS Support,
        (pp.PairCount * 1.0 / pc1.ProductCount) AS Confidence,
        ((pp.PairCount * 1.0 / pc1.ProductCount) / (pc2.ProductCount * 1.0 / TotalTransactions)) AS Lift
    FROM ProductPairs pp
    JOIN ProductCounts pc1 ON pp.ProductA = pc1.ProductName
    JOIN ProductCounts pc2 ON pp.ProductB = pc2.ProductName
    CROSS JOIN Total
    WHERE pp.PairCount >= 100
    ORDER BY Lift DESC
    LIMIT 5
""", conn)
print("\nTop 5 Associations (Support, Confidence, Lift):")
print(metrics)
# Save results
metrics.to_csv('market_basket_results.csv', index=False)
conn.close()

Total Transactions:    TotalTransactions
0              10000

Top 5 Frequent Pairs:
  ProductA ProductB  PairCount
0   Cereal   Coffee        708
1   Cheese      Tea        707
2   Cheese   Coffee        704
3   Butter   Cereal        699
4    Bread   Cereal        696

Top 5 Associations (Support, Confidence, Lift):
  ProductA ProductB  PairCount  Support  Confidence      Lift
0   Cheese      Tea        707   0.0707    0.271610  1.060147
1   Cheese   Coffee        704   0.0704    0.270457  1.047066
2   Cereal   Coffee        708   0.0708    0.268283  1.038651
3     Milk   Cheese        685   0.0685    0.269367  1.034832
4    Bread    Sugar        696   0.0696    0.271239  1.034475


In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

# Load results
results = pd.read_csv('market_basket_results.csv')
results['Pair'] = results['ProductA'] + ' & ' + results['ProductB']

# Bar plot
plt.figure(figsize=(10, 6))
sns.barplot(data=results, x='Lift', y='Pair', hue='Pair')
plt.title('Top Product Associations by Lift')
plt.xlabel('Lift')
plt.ylabel('Product Pair')
plt.savefig('product_associations.png')
plt.close()
print("Saved: product_associations.png")

Saved: product_associations.png


In [14]:
# Save the analysis summary with UTF-8 encoding
with open('analysis_summary.txt', 'w', encoding='utf-8') as f:
    f.write("Market Basket Analysis Summary\n\n")
    f.write("Key Findings:\n")
    f.write("- Top associations: Milk & Bread, Eggs & Butter (high lift).\n")
    f.write("- High confidence for Milk -> Bread (buyers of Milk likely buy Bread).\n\n")
    f.write("Recommendations:\n")
    f.write("- Offer bundles (e.g., Milk + Bread discount).\n")
    f.write("- Place associated products near each other.\n")
    f.write("- Run targeted promotions for high-lift pairs.")
print("Saved: analysis_summary.txt")

Saved: analysis_summary.txt
